# Implementing a basic optimizer and learner

In [1]:
!pip install -Uqq fastai

from fastai.vision.all import *

In [2]:
class MyOptimizer:
    def __init__(self, params, lr): 
        self.params, self.lr = list(params), lr

    def step(self, *args, **kwargs):
        for p in self.params: 
            p.data -= p.grad.data * self.lr

    def zero_grad(self, *args, **kwargs):
        for p in self.params: 
            p.grad = None

In [3]:
class MyLearner:
    def __init__(self, model, dset_train, dset_valid, batch_size, loss, metric, lr=0.1):
        self.model = model
        self.optimizer = MyOptimizer(model.parameters(), lr)
        self.loss = loss
        self.metric = metric
        
        self.dl_train = DataLoader(dset_train, batch_size=batch_size)
        self.dl_valid = DataLoader(dset_valid, batch_size=batch_size)

    def calc_grad(self, xb, yb):
        preds = self.model(xb)
        loss = self.loss(preds, yb)
        loss.backward()
        
    def train_epoch(self):
        for xb, yb in self.dl_train:
            self.calc_grad(xb, yb)
            self.optimizer.step()
            self.optimizer.zero_grad()

    def validate_epoch(self):
        metrics = [self.metric(self.model(xb), yb) for xb, yb in self.dl_valid]
#         return(metrics)
        return round(torch.stack(metrics).mean().item(), 5)

    def train(self, epochs):
        for i in range(epochs):
            self.train_epoch()
            print(self.validate_epoch(), end='  ')

Dummy model

In [4]:
linear_model = nn.Linear(10, 1)

def loss(predictions, targets):
    return ((predictions - targets)**2).mean().sqrt()

def dev(predictions, targets):
    return ((predictions - targets)/targets).abs().mean()

Dummy data

In [5]:
# random numbers from -5 to 5, 50 sets of 10
data = torch.rand(50, 10) * 10 - 5

# labels with linear function + noise
real_weights = torch.unsqueeze(torch.tensor([1., -1., 0., 2., -2., 0., -1., 1., 0., 3.]), dim=1)
labels = torch.randn(50, 10) / 3 + data @ real_weights - 3

# train/test split
dset_train = list(zip(data[:40], labels[:40]))
dset_valid = list(zip(data[40:], labels[40:]))

Train dummy model

In [6]:
learn = MyLearner(linear_model, dset_train, dset_valid, 5, loss, dev, lr=0.05)
learn.train(30)
learn.optimizer.params

0.98198  0.68749  0.46389  0.36567  0.32907  0.27423  0.22832  0.17661  0.12628  0.09147  0.0771  0.07276  0.07074  0.06807  0.07308  0.0782  0.07735  0.07751  0.07789  0.0779  0.07793  0.07796  0.07797  0.07797  0.07798  0.07798  0.07798  0.07798  0.07798  0.07798  

[Parameter containing:
 tensor([[ 1.0333, -0.8684,  0.0332,  2.0888, -2.0157, -0.0562, -1.0531,  0.9266,
          -0.0200,  3.0455]], requires_grad=True),
 Parameter containing:
 tensor([-2.9286], requires_grad=True)]